## 指数估值择时策略

In [ ]:
import pandas as pd
import numpy as np
# 初始化函数,全局只运行一次
def init(context):
    # 设置要操作的指数
    context.security = '000001.SH'
    # 设置基准收益：上证指数
    set_benchmark(context.security)
    # 每周第一个交易日运行
    run_monthly(trade,1)
    # 估值的年限
    context.year = 5
    # 估值序列储存
    context.pelist = []
## 开盘时运行函数
def trade(context, bar_dict):
    # 获取时间
    date =  get_last_datetime().strftime('%Y-%m-%d')
    # 获取估值
    pe_rate = get_index_valuation(context,date,context.security)
    # 加入序列
    context.pelist.append(pe_rate)
    # 判定估值信号
    if len(context.pelist)>1 and context.pelist[-2]<20 and context.pelist[-1]>=20:
        # 全仓
        order_target_percent(context.security,1)
        # 打印
        print('买入指数{}:当前估值百分位{}'.format(context.security,pe_rate))
    elif len(context.pelist)>1 and context.pelist[-2]>80 and context.pelist[-1]<=80:
        # 空仓
        order_target(context.security,0)
        # 打印
        print('卖出指数{}:当前估值百分位{}'.format(context.security,pe_rate))
# 创建函数：获取指数估值水平
def get_index_valuation(context,date,index_code):
    '''
    输出：市盈率
    '''
    # 创建dataframe
    datadf = pd.DataFrame(columns = ['PE'])
    # 创建月初交易日列表
    monthdatelist = []
    # 获取日期
    datelist = get_trade_days(None,date,context.year*250).strftime('%Y%m%d')
    # 循环时间序列
    for t in range(1,len(datelist)):
        # 判断月度
        if datelist[t-1][4:6]!=datelist[t][4:6]:
            # 添加新月度日期
            monthdatelist.append(datelist[t])
    # 循环月度日期
    for date in monthdatelist[-context.year*12:]:
        # 调用指数估值指标计算函数
        datadf.loc[date] = get_index_PB_PE_PB1(date,index_code)
    # 最大值   
    values_max = datadf.max()
    # 最小值
    values_min = datadf.min()
    # 当前值
    values_now = datadf.ix[-1]
    # 百分位
    T = (values_now - values_min)/(values_max - values_min)
    # 输出
    return round(T.PE,4)*100
# 创建函数：指数估值指标计算
def get_index_PB_PE_PB1(date,indexcode):
    '''
    函数使用注释：
    输入：日期，指数代码
    算法：中位数
    输出：指数市盈率
    '''
    # 获取成份股
    stocks = get_index_stocks(indexcode,date)
    # 获取市盈率数据
    q = query(factor.symbol,
              factor.pe_ttm).filter(
              factor.symbol.in_(stocks),
              factor.date == date)
    # 获取数据
    df = get_factors(q)
    # 计算中位数
    PE_index = df.median().factor_pe_ttm
    return [PE_index]